In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [45]:
from sklearn.model_selection import train_test_split
import folium
from folium.plugins import FastMarkerCluster
import re
from datetime import datetime


In [3]:
df = pd.read_csv("../data/listings.csv.gz",compression="gzip")

In [4]:
df.shape

(8739, 75)

## 1) Splitting the dataset into 70-15-15 ratio

In [5]:
df_full_train,df_test = train_test_split(df,test_size=0.15,random_state=810)
df_train,df_df_val = train_test_split(df_full_train,test_size=0.15,random_state=810)

In [6]:
print("Dataset shape when train and validation is combined",df_full_train.shape)
print("Dataset shape of the test dataset",df_test.shape)

Dataset shape when train and validation is combined (7428, 75)
Dataset shape of the test dataset (1311, 75)


## 2) Data Preprocessing

In [7]:
df_full_train.head().T

,5106,5961,7311,1213,2387
id,1020339493261692070,991906029869083109,800065755352436194,1012266793074500896,29923896
listing_url,https://www.airbnb.com/rooms/1020339493261692070,https://www.airbnb.com/rooms/991906029869083109,https://www.airbnb.com/rooms/800065755352436194,https://www.airbnb.com/rooms/1012266793074500896,https://www.airbnb.com/rooms/29923896
scrape_id,20231212015436,20231212015436,20231212015436,20231212015436,20231212015436
last_scraped,2023-12-12,2023-12-12,2023-12-12,2023-12-12,2023-12-12
source,city scrape,previous scrape,city scrape,city scrape,city scrape
...,...,...,...,...,...
calculated_host_listings_count,2,1,1,1,3
calculated_host_listings_count_entire_homes,2,1,1,1,0
calculated_host_listings_count_private_rooms,0,0,0,0,3
calculated_host_listings_count_shared_rooms,0,0,0,0,0


In [8]:
df_full_train.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [9]:
cat_features = list((df_full_train.dtypes[df_full_train.dtypes=="object"]).index)
print(len(cat_features))
print(cat_features)

34
['listing_url', 'last_scraped', 'source', 'name', 'neighborhood_overview', 'picture_url', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 'bathrooms_text', 'amenities', 'price', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'license', 'instant_bookable']


In [10]:
df_full_train.columns = df_full_train.columns.str.replace(" ","_").str.lower()
for col in cat_features:
    df_full_train[col] = df_full_train[col].str.replace(" ","_").str.lower()

In [11]:
all_features = list(df_full_train.columns)
print(len(all_features))

75


In [12]:
for col in all_features:
    print(col)
    print(df_full_train[col].nunique())
    print(df_full_train[col].unique()[:5])
    print("----------------------------------------------------------------------------------")

id
7428
[1020339493261692070  991906029869083109  800065755352436194
 1012266793074500896            29923896]
----------------------------------------------------------------------------------
listing_url
7428
['https://www.airbnb.com/rooms/1020339493261692070'
 'https://www.airbnb.com/rooms/991906029869083109'
 'https://www.airbnb.com/rooms/800065755352436194'
 'https://www.airbnb.com/rooms/1012266793074500896'
 'https://www.airbnb.com/rooms/29923896']
----------------------------------------------------------------------------------
scrape_id
1
[20231212015436]
----------------------------------------------------------------------------------
last_scraped
2
['2023-12-12' '2023-12-13']
----------------------------------------------------------------------------------
source
2
['city_scrape' 'previous_scrape']
----------------------------------------------------------------------------------
name
3256
['rental_unit_in_amsterdam_·_★new_·_1_bedroom_·_1_bed_·_1_bath'
 'rental_unit_in_ams

In [13]:
#Few columns have information regarding scrap so they are irrelevant to the problem statement.
#columns related to scrap are :  scrape_id, last_scraped, source, calendar_last_scraped

In [14]:
#Rows having alot of unique value and does not contain much information regarding the pricing of the house are irrelevant as well.
#Columns are : id, listing_url, name, description, picture_url, host_id, host_url, host_name, host_location, host_thumbnail_url, host_picture_url, host_verifications, host_has_profile_pic, host_identity_verified,neighbourhood_group_cleansed, amenities
                #bathrooms,minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, calendar_updated
                #has_availability, availability_30, availability_60, availability_90, availability_365, first_review, last_review, license, calculated_host_listings_count, calculated_host_listings_count_entire_homes,
                #calculated_host_listings_count_private_rooms, calculated_host_listings_count_shared_rooms

In [15]:
#Columns that might be interesting but not sure to be used or not
#Columns : neighborhood_overview, host_about, host_response_rate, host_neighbourhood, host_listings_count, host_total_listings_count, property_type, maximum_nights, number_of_reviews_ltm, number_of_reviews_l30d,
            #review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, instant_bookable,  minimum_nights,host_acceptance_rate, reviews_per_month, 

In [16]:
#Columns to be used to model the problem statement
#columns: host_since, host_is_superhost, neighbourhood_cleansed, latitude, longitude, room_type, accommodates, bathrooms_text, price, number_of_reviews, review_scores_rating  

In [32]:
columns = ["host_since", "host_is_superhost","host_total_listings_count","neighbourhood_cleansed", "latitude", "longitude", "room_type", "accommodates", "bathrooms_text", "beds", "price", "number_of_reviews", "review_scores_rating","availability_365","minimum_nights"]
df_filter_full_train = df_full_train[columns]

In [33]:
df_filter_full_train.head().T

,5106,5961,7311,1213,2387
host_since,2023-07-28,2014-10-23,2014-04-08,2015-05-26,2017-10-30
host_is_superhost,f,f,f,f,t
host_total_listings_count,2,1,1,8,5
neighbourhood_cleansed,ijburg_-_zeeburgereiland,centrum-oost,de_baarsjes_-_oud-west,oud-oost,centrum-west
latitude,52.361914,52.36876,52.364833,52.351616,52.37988
longitude,4.988083,4.928422,4.876748,4.912425,4.88459
room_type,entire_home/apt,entire_home/apt,entire_home/apt,entire_home/apt,private_room
accommodates,2,2,2,3,2
bathrooms_text,1_bath,1.5_baths,1_bath,1_bath,1_private_bath
beds,1.0,1.0,1.0,2.0,1.0


In [34]:
print("Number of duplicate values in the dataset are", df_filter_full_train.duplicated().sum())
df_filter_full_train.drop_duplicates(inplace=True)

Number of duplicate values in the dataset are 1


In [35]:
df_filter_full_train.isna().sum()

host_since                     0
host_is_superhost             42
host_total_listings_count      0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
room_type                      0
accommodates                   0
bathrooms_text                 6
beds                          61
price                        254
number_of_reviews              0
review_scores_rating         778
availability_365               0
minimum_nights                 0
dtype: int64

#### 2.1) Cleaning "price" column

In [36]:
#Cleaning price column
print("Number of NaN values are ", df_filter_full_train.price.isna().sum())
df_filter_full_train.dropna(subset=['price'], inplace=True)
def remove_dollar_sign(text):
    text = str(text)
    numeric_text = text.replace("$","").replace(",","")
    return float(numeric_text)
df_filter_full_train.price =  df_filter_full_train["price"].apply(remove_dollar_sign)

Number of NaN values are  254


#### 2.2) Cleaning "bathroom_text" column

In [37]:
df_filter_full_train.bathrooms_text.isna().sum()

6

In [38]:
print("Number of bed rooms count per bathroom number \n",df_filter_full_train.groupby("beds").bathrooms_text.value_counts())
print("-------------------------------------------------------------------------------------------------------------------------")
print("Number of beds in the Airbnb listing for which bathroom text is missing \n",df_filter_full_train[df_filter_full_train.bathrooms_text.isna()]["beds"])

Number of bed rooms count per bathroom number 
 beds  bathrooms_text  
1.0   1_bath              2132
      1.5_baths            927
      1_private_bath       408
      1_shared_bath        200
      1.5_shared_baths     109
                          ... 
22.0  2.5_baths              1
24.0  5.5_baths              3
26.0  2.5_baths              1
30.0  17_baths               1
33.0  3_baths                1
Name: count, Length: 102, dtype: int64
-------------------------------------------------------------------------------------------------------------------------
Number of beds in the Airbnb listing for which bathroom text is missing 
 312     1.0
8173    1.0
3845    1.0
2811    1.0
8319    3.0
5611    4.0
Name: beds, dtype: float64


In [39]:
#Since the missing bathrooms text have smaller number of beds, hence number of beds is being filled in missing bathroom text
df_filter_full_train["bathrooms_text"].fillna(str(df_filter_full_train["beds"]), inplace=True)

In [40]:
#creating a new columns to check if the bathroom is shared or not and converting bathrooms_text into a float columns
def bathroom_number(text):
    text = str(text)
    if "half" in text:
        return 0.5
    else:
        number = re.findall(r'\d+\.\d+|\d+', text)
    return float(number[0])
df_filter_full_train["bathrooms"] = df_filter_full_train["bathrooms_text"].apply(bathroom_number)
df_filter_full_train["private"] = df_filter_full_train["bathrooms_text"].apply(lambda x:0 if "shared" in str(x) else 1)

#### 2.3) Cleaning "host" columns

In [41]:
# Filling NAN values in super host column as normal host
df_filter_full_train.host_is_superhost.fillna("f",inplace = True)


In [44]:
# Changing the "host since" column to number of days from today.
df_filter_full_train[] = df_filter_full_train.host_since

dtype('O')

## 3)EDA 

#### 3.1) Neighbourhoods

In [ ]:
nei_frequency = df_full_train["neighbourhood_cleansed"].value_counts().sort_values(ascending=True)
nei_frequency.plot.barh(figsize=(10,8),color = "b", width=0.5)
plt.title("Number of listings by neighbourhood", fontsize=15)
plt.xlabel("Number of listings",fontsize=10)
plt.show()

In [ ]:
lats = df_full_train["latitude"].tolist()
lons = df_full_train["longitude"].tolist()
locations = list(zip(lats,lons))
map1 = folium.Map(location=[52.38, 4.9],zoom_start=11.5)
FastMarkerCluster(data=locations).add_to(map1)
map1

In [ ]:
#most of the places are location near to the city centre

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(df_full_train.longitude,df_full_train.latitude,hue=df_full_train.neighbourhood_cleansed)
plt.ioff()

In [ ]:
feq = df_full_train[df_full_train['accommodates']==2]
feq = feq.groupby('neighbourhood_cleansed')["price"].mean().sort_values(ascending=True)
feq.plot.barh(figsize=(10, 8), color='b', width=0.4)
plt.title("Average daily price for a 2-persons accommodation", fontsize=20)
plt.xlabel('Average daily price (Euro)', fontsize=12)
plt.ylabel("")
plt.show()

#### 2.2) Room and property types

In [ ]:
room_fre = df_full_train["room_type"].value_counts().sort_values(ascending=True)
room_fre.plot.barh(figsize=(15,3),width=0.5,color=["g","b","r"])
plt.xlabel("Number for listings for each type pf room")
plt.title("Room type listing in each category")
plt.show()

In [ ]:
#Entire home/apt has the most listing while shared room and hotel room has hardly booked through airbnb so it has less listings

In [ ]:
df_full_train.bathrooms_text.value_counts()

In [ ]:
df_full_train.groupby("room_type").bathrooms_text.value_counts()

In [ ]:
#create new columns if the bathroom is shared or not and converting bathrooms_text into a float columns

In [ ]:
#since the beds are lower values only for which bathrooms_text is missing so bed column value is filled in the bathroom columns 
df_full_train["bathrooms_text"].fillna(df_full_train["beds"], inplace=True)

In [ ]:
def bathroom_number(text):
    text = str(text)
    if "half" in text:
        return 0.5
    else:
        number = re.findall(r'\d+\.\d+|\d+', text)
    return float(number[0])
df_full_train["bathrooms"] = df_full_train["bathrooms_text"].apply(bathroom_number)
df_full_train["private"] = df_full_train["bathrooms_text"].apply(lambda x:0 if "shared" in str(x) else 1)

In [ ]:
df_full_train["private"] = df_full_train["bathrooms_text"].apply(lambda x:0 if "shared" in str(x) else 1)

In [ ]:
df_full_train.private.value_counts().sort_values(ascending=True).plot.barh(figsize=(10,3),width=0.3,color=["g","b"])


In [ ]:
bathroom_counts = df_full_train["bathrooms"].value_counts().sort_index()
# Plot the horizontal bar chart
plt.figure(figsize=(15, 3))
bars = bathroom_counts.plot.barh(width=0.5, color='b')
plt.xlabel('Count')
plt.ylabel('Number of Bathrooms')
plt.title('Distribution of Bathrooms')

# Annotate each bar with its value
for bar in bars.patches:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{int(bar.get_width())}', 
             va='center', ha='left', fontsize=8, color='black')

plt.grid(axis='x', linestyle='--', alpha=0.7)  # Add vertical grid lines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
accommodation_counts = df_full_train["accommodates"].value_counts().sort_index()
# Plot the horizontal bar chart
plt.figure(figsize=(15, 3))
bars = accommodation_counts.plot.barh(width=0.5, color='b')
plt.xlabel('Count')
plt.ylabel('Accommodation size')
plt.title('Distribution of Accommodation')

# Annotate each bar with its value
for bar in bars.patches:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{int(bar.get_width())}', 
             va='center', ha='left', fontsize=8, color='black')

plt.grid(axis='x', linestyle='--', alpha=0.7)  # Add vertical grid lines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
ax = df_full_train["minimum_nights"].value_counts().sort_index().plot(kind='bar', color='b')
plt.xlabel('Minimum Nights')
plt.ylabel('Frequency')
plt.title('Distribution of Minimum Nights')
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add horizontal grid lines

# Annotate each bar with its count value
for i in ax.patches:
    plt.text(i.get_x() + i.get_width()/2, i.get_height() + 10, str(int(i.get_height())), 
             ha='center', va='bottom', fontsize=8)

plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()
